In [4]:
from typing import Any, List, Union, Dict
from pydantic import BaseModel, field_validator


class SingleTypeMixin:
    """Mixin to enforce only single float values for `param1` and `param2`."""

    @field_validator("param1", "param2", mode="before")
    @classmethod
    def enforce_single_type(cls, value):
        if isinstance(value, list):
            raise ValueError("Lists are not allowed for this class.")
        if not isinstance(value, (float, int)):  # Allowing int to be coerced to float
            raise ValueError("Must be a float.")
        return float(value)


class MultiTypeMixin:
    """Mixin to allow both single float and list of floats for `param1` and `param2`."""

    @field_validator("param1", "param2", mode="before")
    @classmethod
    def allow_list_type(cls, value):
        if isinstance(value, (float, int)):
            return float(value)  # Convert int to float if necessary
        if isinstance(value, list) and all(isinstance(v, (float, int)) for v in value):
            return [float(v) for v in value]  # Convert all ints to floats
        raise ValueError("Must be a float or a list of floats.")
    

def validate_nested_single(cls):
    """Ensures nested attributes (including inside dictionaries) only contain floats."""
    @field_validator("nested", mode="before")
    @classmethod
    def enforce_nested_single(cls, value):
        if isinstance(value, BaseModel):  # Validate Pydantic object
            for field, field_value in value.model_dump().items():
                if isinstance(field_value, list):
                    raise ValueError(f"Nested attribute '{field}' must not be a list.")
                if not isinstance(field_value, (float, int)):
                    raise ValueError(f"Nested attribute '{field}' must be a float.")

        elif isinstance(value, dict):  # Validate dictionary contents
            for key, dict_value in value.items():
                if isinstance(dict_value, BaseModel):  # Recursively validate BaseModel objects
                    for field, field_value in dict_value.model_dump().items():
                        if isinstance(field_value, list):
                            raise ValueError(f"Nested dictionary attribute '{key}.{field}' must not be a list.")
                        if not isinstance(field_value, (float, int)):
                            raise ValueError(f"Nested dictionary attribute '{key}.{field}' must be a float.")

        return value

    cls.model_rebuild()  # Rebuild model after adding validator
    return cls


def validate_nested_multi(cls):
    """Ensures nested attributes (including inside dictionaries) allow floats and lists of floats."""
    @field_validator("nested", mode="before")
    @classmethod
    def enforce_nested_multi(cls, value):
        if isinstance(value, BaseModel):  # Validate Pydantic object
            for field, field_value in value.model_dump().items():
                if not (isinstance(field_value, (float, int)) or
                        (isinstance(field_value, list) and all(isinstance(v, (float, int)) for v in field_value))):
                    raise ValueError(f"Nested attribute '{field}' must be a float or list of floats.")

        elif isinstance(value, dict):  # Validate dictionary contents
            for key, dict_value in value.items():
                if isinstance(dict_value, BaseModel):  # Recursively validate BaseModel objects
                    for field, field_value in dict_value.model_dump().items():
                        if not (isinstance(field_value, (float, int)) or
                                (isinstance(field_value, list) and all(isinstance(v, (float, int)) for v in field_value))):
                            raise ValueError(f"Nested dictionary attribute '{key}.{field}' must be a float or list of floats.")

        return value

    cls.model_rebuild()  # Rebuild model after adding validator
    return cls


class BaseTemplate(BaseModel):
    """Base simulation model that contains a generic nested object."""
    param1: Union[float, List[float]]
    param2: Union[float, List[float]]
    nested: Any  # Will be validated dynamically in subclasses


@validate_nested_single
class Simulation(SingleTypeMixin, BaseTemplate):
    """Only allows single float values and ensures nested attributes follow the same rule."""
    pass


@validate_nested_multi
class SimulationTemplate(MultiTypeMixin, BaseTemplate):
    """Allows both single float and list of floats, ensuring nested attributes follow the same rule."""
    pass


class Stimulus(BaseModel):
    """A nested model containing numeric attributes."""
    nested_param1: Union[float, List[float]]
    nested_param2: Union[float, List[float]]






# Valid Simulation instance (SingleType) with direct nested object
nested_single = Stimulus(nested_param1=1.5, nested_param2=2.5)  # ✅ Must be single floats
simulation = Simulation(param1=2.0, param2=3.0, nested=nested_single)
print(simulation)

# Valid SimulationTemplate instance (MultiType) with direct nested object
nested_multi = Stimulus(nested_param1=[1.5, 2.5], nested_param2=3.5)  # ✅ Allows lists
simulation_template = SimulationTemplate(param1=[2.0, 4.0], param2=3.0, nested=nested_multi)
print(simulation_template)

# Valid Simulation with nested dictionary containing BaseModel
simulation_with_dict = Simulation(
    param1=2.0, 
    param2=3.0, 
    nested={"config": Stimulus(nested_param1=2.2, nested_param2=4.4)}
)
print(simulation_with_dict)

# Valid SimulationTemplate with nested dictionary containing BaseModel
simulation_template_with_dict = SimulationTemplate(
    param1=[2.0, 4.0], 
    param2=3.0, 
    nested={"config": Stimulus(nested_param1=[1.1, 2.2], nested_param2=3.3)}
)
print(simulation_template_with_dict)




param1=2.0 param2=3.0 nested=Stimulus(nested_param1=1.5, nested_param2=2.5)
param1=[2.0, 4.0] param2=3.0 nested=Stimulus(nested_param1=[1.5, 2.5], nested_param2=3.5)
param1=2.0 param2=3.0 nested={'config': Stimulus(nested_param1=2.2, nested_param2=4.4)}
param1=[2.0, 4.0] param2=3.0 nested={'config': Stimulus(nested_param1=[1.1, 2.2], nested_param2=3.3)}


In [ ]:
# ❌ Nested dictionary contains a BaseModel with a list (not allowed in Simulation)
try:
    invalid_simulation_dict = Simulation(
        param1=2.0, 
        param2=3.0, 
        nested={"config": Stimulus(nested_param1=[1.5, 2.5], nested_param2=3.5)}
    )
except ValueError as e:
    print(e)

# ❌ Nested dictionary contains a BaseModel with a string (not allowed in SimulationTemplate)
try:
    invalid_simulation_template_dict = SimulationTemplate(
        param1=[2.0, 4.0], 
        param2=3.0, 
        nested={"config": Stimulus(nested_param1="invalid", nested_param2=3.5)}
    )
except ValueError as e:
    print(e)